# Beam center finder

Description of the beam center finding algorithm.

In [ ]:
import scipp as sc
from ess import loki, sans
from ess.logging import configure_workflow
import scippneutron as scn
import numpy as np

import plopp as pp
pp.patch_scipp()

In [ ]:
logger = configure_workflow('sans_beam_center_finder', filename='sans.log')

In [ ]:
# Load the data file
sample = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.nxs'))
sample

By making a simple image from the detector panel,
summing along the `tof` dimension,
we can see that the signal from the beam is not in the centre of the panel (marked by the red dot).
The centre of the beam is the dark circular region (create by the beam stop),
to the lower right of the red dot, surrounded by a faint blurry ring of higher counts. 

In [ ]:
sample.coords['x'] = sample.coords['position'].fields.x
sample.coords['y'] = sample.coords['position'].fields.y
image = sample.bin(y=120, x=128).sum('tof')
p = image.plot(norm='log', aspect='equal')
p.ax.plot(0, 0, 'o', color='red', ms=5)
p

## Description of the procedure

The prodecure to determine the precise location of the beam center is the following:

1. obtain an initial guess by computing the center-of-mass of the pixels, weighted by the counts on each pixel
2. from that initial guess, divide the panel into 4 quadrants
3. compute $I(Q)$ inside each quadrant and compute the residual difference between all 4 quadrants
4. iteratively move the centre position and repeat 2. and 3. until all 4 $I(Q)$ curves lie on top of each other

## Initial guess: center-of-mass calculation

Computing the center-of-mass is straightforward using the vector `position` coordinate.

In [ ]:
# First sum the data along the 'tof' dimension
summed = sample.sum('tof')

# The weights are just the data counts in each pixel
weights = sc.values(summed.data)

# The center-of-mass is simply the weighted mean of the positions
com = sc.sum(summed.coords['position'] * weights) / weights.sum()
xc = com.fields.x
yc = com.fields.y

We can now plot the center-of-mass on the same image as before:

In [ ]:
p = image.plot(norm='log', aspect='equal')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
p

## Making 4 quadrants

We divide the panel into 4 quadrants.
Because we will be tweaking the position of the center along the horizontal (`x`) and vertical (`y`) axes,
we place diagonal boundaries for the quadrants, instead of aligning them with the `x` and `y` axes.

In the following plot, we slightly alter the count values to visualize the 4 groups of pixels.

In [ ]:
pi = sc.constants.pi.value * sc.units.rad

phi = sc.atan2(y=sc.midpoints(image.coords['y'] - yc),
               x=sc.midpoints(image.coords['x'] - xc))
phi_offset = pi / 4
phi = (phi + phi_offset) % (2 * pi)

image.data = image.data * ((phi / (2 * phi_offset)).astype(int) + 1.0)
p = image.plot(norm='log', aspect='equal')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
dx = 0.3
p.ax.text(xc.value + dx, yc.value, 'Right', ha='center', va='center')
p.ax.text(xc.value, yc.value + dx, 'Top', ha='center', va='center')
p.ax.text(xc.value - dx, yc.value, 'Left', ha='center', va='center')
p.ax.text(xc.value, yc.value - dx, 'Bottom', ha='center', va='center')
p

## Adding a circular mask

It is evident from the figure above that some quadrants (e.g. `Top` and `Left`) contains more pixels that others.
They also extend further away from the center, which means that more pixels can contribute to a given $Q$ bin.
To avoid introducing such bias when searching for the beam center, we add a circular mask onto the detector panel.

In [ ]:
masking_radius = sc.scalar(0.35, unit='m')

r = sc.sqrt(sc.midpoints(image.coords['x'] - xc)**2 +
            sc.midpoints(image.coords['y'] - yc)**2)
image.masks['circle'] = r > masking_radius

p = image.plot(norm='log', aspect='equal')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
p

## Converting to $Q$ inside each quadrant

We now use Scipp's `transform_coords` to convert the data to $Q$, and sum the counts (using `groupby`) inside each quadrant.
In this example, we include the effects of gravity in our calculation for $Q$.

In [ ]:
from scipp.constants import g
sample.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g
# Create the coordinate transformation graph
graph, _ = sans.i_of_q.make_coordinate_transform_graphs(gravity=True, scatter=True)

We now define a function which will apply the center offset to the pixel coordinates,
compute $Q$, and group the data counts into 4 phi bins.

In [ ]:
def to_q(xy, sample, graph, q_bins, masking_radius):
    # Make a copy of the original data
    data = sample.copy(deep=False)
    data.coords['position'] = data.coords['position'].copy(deep=True)
    # Offset the position according to the initial guess from the center-of-mass
    u = data.coords['position'].unit
    data.coords['position'].fields.x -= sc.scalar(xy[0], unit=u)
    data.coords['position'].fields.y -= sc.scalar(xy[1], unit=u)
    # Add the circular mask
    r = sc.sqrt(data.coords['position'].fields.x**2 +
                data.coords['position'].fields.y**2)
    data.masks['m'] = r > masking_radius

    # Convert to Q
    da_q = data.transform_coords('Q', graph=graph)
    # Request phi coordinate from transformation graph
    da_phi = da_q.transform_coords('phi', graph=graph)
    phi_offset = pi / 4
    da_phi.coords['phi'] += phi_offset
    # Histogram in Q
    da_h = da_phi.hist(Q=q_bins)
    # Group by phi
    phi_bins = sc.linspace('phi', 0, np.pi * 2, 5, unit='rad') + phi_offset
    return sc.groupby(da_h, group='phi', bins=phi_bins).sum('spectrum')

In [ ]:
# Define Q binning
q_bins = sc.linspace('Q', 0.02, 0.3, 101, unit='1/angstrom')

grouped = to_q([xc.value, yc.value],
               sample=sample,
               graph=graph,
               q_bins=q_bins,
               masking_radius=masking_radius)
grouped

In [ ]:
pp.plot(sc.collapse(grouped, keep='Q'), norm='log')

As we can see, the overlap between the curves from the 4 quadrants is not satisfactory.
We will now use an iterative procedure to improve our initial guess, until a good overlap between the curves is found.

For this, we first define a cost function, which gives us an idea of how good the overlap is.

In [ ]:
def cost(xy, sample, graph, q_bins, masking_radius):
    da = to_q(xy,
              sample=sample,
              graph=graph,
              q_bins=q_bins,
              masking_radius=masking_radius)
    ref = da['phi', 0]
    cost = ((da['phi', 1] - ref)**2 + (da['phi', 2] - ref)**2 +
            (da['phi', 3] - ref)**2) / ref**2
    return cost.sum().value

Next, we use Scipy's `minimize` utility from the `optimize` module, to iteratively minimize the computed cost.

In [ ]:
from scipy.optimize import minimize

# The minimizer works best if given bounds, which are the bounds of our detector panel
x = sample.coords['position'].fields.x
y = sample.coords['position'].fields.y
res = minimize(cost,
               x0=[xc.value, yc.value],
               args=(sample, graph, q_bins, masking_radius),
               bounds=[(x.min().value, x.max().value),
                       (y.min().value, y.max().value)],
               method='Nelder-Mead',
               tol=0.1)
res

Once the iterations completed, the returned object contains the best estimate for the beam center:

In [ ]:
res.x

We can now feed this value again into our `to_q` function, to inspect the $Q$ intensity in all 4 quadrants:

In [ ]:
grouped = to_q(res.x,
               sample=sample,
               graph=graph,
               q_bins=q_bins,
               masking_radius=masking_radius)

pp.plot(sc.collapse(grouped, keep='Q'), norm='log')

The overlap between the curves is excellent, allowing us to safely perform an azimuthal summation of the counts around the beam center.

As a consistency check, we plot the refined beam center position onto the detector panel image:

In [ ]:
sample.coords['x'] = sample.coords['position'].fields.x
sample.coords['y'] = sample.coords['position'].fields.y
image = sample.bin(y=120, x=128).sum('tof')
p = image.plot(norm='log', aspect='equal')
p.ax.plot(res.x[0], res.x[1], 'o', color='red', ms=5)
p